In [ ]:
import time
import torch
import wandb

def log_model_performance(encoder, batch_tensor, label: str, runs: int = 100):
    device = batch_tensor.device
    encoder.eval()

    # Warm-up
    with torch.no_grad():
        _ = encoder(batch_tensor)

    torch.cuda.synchronize(device)
    total_time = 0
    batch_size = batch_tensor.size(0)   

    with torch.no_grad():
        for step in range(runs):
            start = time.time()
            _ = encoder(batch_tensor)
            torch.cuda.synchronize(device)
            end = time.time()
            total_time = (end - start)
            throughput = batch_size / total_time

            wandb.log({
                f"throughput_fps/{label}": throughput
            },step=step)

In [ ]:
import sys
sys.path.append('./src')

# Logging MobileNetV3-Large

In [ ]:
from mobile_gemma import MobileNetImageEncoder, MobileNetConfig
encoder = MobileNetImageEncoder(config=MobileNetConfig())

In [ ]:
import torch

batch_size = 32
channels = 3
height = 224
width = 224

dummy_batch = torch.rand(batch_size, channels, height, width).to("cuda")
encoder.eval().to("cuda")
import wandb
wandb.init(
    project="Final-Project",
    name="mobilenetv3_run",
    config={"encoder": "mobilenetv3", "batch_size": batch_size}
)

In [5]:
log_model_performance(encoder=encoder,batch_tensor=dummy_batch,label="mobilenetv3_large")

# Logging Siglip

In [ ]:
from siglip import SiglipVisionModel, SiglipVisionConfig
import torch

batch_size = 32
channels = 3
height = 224
width = 224

dummy_batch = torch.rand(batch_size, channels, height, width).to("cuda")
siglip_encoder = SiglipVisionModel(config=SiglipVisionConfig())
siglip_encoder.eval().to("cuda")
import wandb
wandb.init(
    project="Final-Project",
    name="siglip_run",
    config={"encoder": "siglip", "batch_size": batch_size}
)

In [ ]:
log_model_performance(encoder=siglip_encoder,batch_tensor=dummy_batch,label="siglip")